In [1]:
import cv2
import os
import timm
import numpy as np
import pandas as pd
import albumentations as A

from glob import glob
from tqdm import tqdm
from easydict import EasyDict
from albumentations.pytorch import ToTensorV2
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.metrics import f1_score

import torch
from torch import nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torch.optim.lr_scheduler import CosineAnnealingLR

In [2]:
class CustomDataset(Dataset):
    def __init__(self, img_list, label_list=None, transforms=None, mode="train") :
        self.img_list = img_list
        
        if mode == "train" : 
            self.label_list = self.label_encoder(label_list)
            
        self.transforms = transforms
        self.mode = mode
    def __len__(self):
        return len(self.img_list)
    
    def __getitem__(self, idx):
        img_path = self.img_list[idx]

        img = cv2.imread(img_path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        
        if self.transforms:            
            img = self.transforms(image=img)['image']
        
        if self.mode == "train" :
            label = self.label_list[idx]
            return img, torch.tensor(label)
        
        elif self.mode == "test" :
            return img
    
    def label_encoder(self, label_list) :
        label_enc = {k : i for i, k in enumerate(sorted(list(set(label_list))))}
        return [label_enc[label] for label in label_list]
    

In [3]:
class CNN(nn.Module) :
    def __init__(self, model_name, num_classes) :
        super(CNN, self).__init__()
        self.model = timm.create_model(model_name=model_name, num_classes=num_classes, pretrained=True)
    
    def forward(self, x) :
        output = self.model(x)
        return output

In [4]:
opt = {
    "test_df_path" : "../data/test_df.csv",
    "train_df_path" : "../data/train_df.csv",
    "submission_df_path" : "../data/sample_submission.csv",
    "img_path" : "../data/test",
    "save_path" : "../data/submission/aug_v2_CEL_15E_0.0249_coat_mini.csv",
    "model_name" : "coat_mini",
    "model_path" : "../model/coat_mini_aug_v2_CEL/15E_0.0249_coat_mini.pt",
    "num_classes" : 88,
    "resize" : 224,
    "device" : "cuda:0",
    "batch_size" : 32
}

opt = EasyDict(opt)

test_transforms = A.Compose([
    A.Normalize(),
    A.Resize(opt.resize, opt.resize),
    ToTensorV2()
])

train_df = pd.read_csv(opt.train_df_path)
labels = list(sorted(train_df['label'].unique()))
label_decoder = {i : k for i, k in enumerate(labels)}

test_df = pd.read_csv(opt.test_df_path)
file_names = list(map(lambda y :os.path.join(opt.img_path, y), test_df['file_name']))

test_data = CustomDataset(file_names, transforms=test_transforms, mode="test")
test_loader = DataLoader(test_data, batch_size=opt.batch_size, shuffle=False)

model = CNN(opt.model_name, opt.num_classes).to(opt.device)
model_data = torch.load(opt.model_path)
model.load_state_dict(model_data["model_state_dict"])
model.eval()

CNN(
  (model): CoaT(
    (patch_embed1): PatchEmbed(
      (proj): Conv2d(3, 152, kernel_size=(4, 4), stride=(4, 4))
      (norm): LayerNorm((152,), eps=1e-05, elementwise_affine=True)
    )
    (patch_embed2): PatchEmbed(
      (proj): Conv2d(152, 216, kernel_size=(2, 2), stride=(2, 2))
      (norm): LayerNorm((216,), eps=1e-05, elementwise_affine=True)
    )
    (patch_embed3): PatchEmbed(
      (proj): Conv2d(216, 216, kernel_size=(2, 2), stride=(2, 2))
      (norm): LayerNorm((216,), eps=1e-05, elementwise_affine=True)
    )
    (patch_embed4): PatchEmbed(
      (proj): Conv2d(216, 216, kernel_size=(2, 2), stride=(2, 2))
      (norm): LayerNorm((216,), eps=1e-05, elementwise_affine=True)
    )
    (cpe1): ConvPosEnc(
      (proj): Conv2d(152, 152, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=152)
    )
    (cpe2): ConvPosEnc(
      (proj): Conv2d(216, 216, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=216)
    )
    (cpe3): ConvPosEnc(
      (proj): Conv2d

In [5]:
with torch.no_grad() :
    answers = []
    for img in tqdm(test_loader) :
        img = img.to(opt.device)
        output = model(img)        
        output = F.softmax(output.cpu())
        
        preds = torch.argmax(output, dim=1)
        
        labels = list(map(lambda x : label_decoder[x.item()], preds))
        answers.extend(labels)            

  0%|                                                                                           | 0/68 [00:00<?, ?it/s]C:\Users\sub\AppData\Local\Temp\ipykernel_6964\2726840510.py:6: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  output = F.softmax(output.cpu())
100%|██████████████████████████████████████████████████████████████████████████████████| 68/68 [01:24<00:00,  1.25s/it]


In [6]:
submission = pd.read_csv(opt.submission_df_path).set_index('index')
submission['label'] = answers
submission

,label
index,
0,tile-glue_strip
1,grid-good
2,transistor-good
3,tile-gray_stroke
4,tile-good
...,...
2149,tile-gray_stroke
2150,screw-good
2151,grid-glue


In [7]:
submission.to_csv(opt.save_path)